In [1]:
import pandas as pd
import numpy as np

import datetime
from datetime import date
import numpy_financial as npf

In [2]:
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline
matplotlib.style.use('ggplot')

In [3]:
#print("Define loan parameters")
#Interest_Rate = 0.0361
#Years = 5
#Payments_Year = 12
#Principal = 15000
#Addl_Principal = 0
start_date = (date(2022,7,1))

In [4]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('JMACData04.xlsx', engine='xlsxwriter')

In [5]:
def amortization_table(interest_rate, years, payments_year, principal, addl_principal=0, start_date=start_date):
    """ Calculate the amortization schedule given the loan details
    
     Args:
        interest_rate: The annual interest rate for this loan
        years: Number of years for the loan
        payments_year: Number of payments in a year
        principal: Amount borrowed
        addl_principal (optional): Additional payments to be made each period. Assume 0 if nothing provided.
                                   must be a value less then 0, the function will convert a positive value to
                                   negative
        start_date (optional): Start date. Will start on first of next month if none provided

    Returns:
        schedule: Amortization schedule as a pandas dataframe
        summary: Pandas dataframe that summarizes the payoff information
    """
    # Ensure the additional payments are negative
    if addl_principal > 0:
        addl_principal = -addl_principal
    
    # Create an index of the payment dates
    rng = pd.date_range(start_date, periods=years * payments_year, freq='MS')
    rng.name = "Payment_Date"
    
    # Build up the Amortization schedule as a DataFrame
    df = pd.DataFrame(index=rng,columns=['Payment', 'Principal', 'Interest', 
                                         'Addl_Principal', 'Curr_Balance'], dtype='float')
    
    # Add index by period (start at 1 not 0)
    df.reset_index(inplace=True)
    df.index += 1
    df.index.name = "Period"
    
    # Calculate the payment, principal and interests amounts using built in Numpy functions
    per_payment = npf.pmt(interest_rate/payments_year, years*payments_year, principal)
    df["Payment"] = per_payment
    df["Principal"] = npf.ppmt(interest_rate/payments_year, df.index, years*payments_year, principal)
    df["Interest"] = npf.ipmt(interest_rate/payments_year, df.index, years*payments_year, principal)
        
    # Round the values
    df = df.round(2) 
    
    # Add in the additional principal payments
    df["Addl_Principal"] = addl_principal
    df["Interest_Rate"] = interest_rate * 100
    
    # Store the Cumulative Principal Payments and ensure it never gets larger than the original principal
    df["Cumulative_Principal"] = (df["Principal"] + df["Addl_Principal"]).cumsum()
    df["Cumulative_Principal"] = df["Cumulative_Principal"].clip(lower=-principal)
    
    df["Cumulative_Interest"] = (df["Interest"]).cumsum()
    df["Cumulative_Interest"] = df["Cumulative_Interest"].clip(lower=-(df["Interest"]).cumsum())
      
    # Calculate the current balance for each period
    df["Curr_Balance"] = principal + df["Cumulative_Principal"]
    
    df["Cumulative_Paid"] = df["Cumulative_Principal"].abs() + df["Cumulative_Interest"].abs()
    df["PctOwed"] = (df["Curr_Balance"] / principal) * 100
    df["PctPrinPaid"] = (df["Cumulative_Principal"] / principal) * -100
    df["PrinipalPct"] = (df["Principal"] / df["Payment"] ) * 100
    df["InterestPct"] = (df["Interest"] / df["Payment"] ) * 100
    
    # Determine the last payment date
    try:
        last_payment = df.query("Curr_Balance <= 0")["Curr_Balance"].idxmax(axis=1, skipna=True)
    except ValueError:
        last_payment = df.last_valid_index()
    
    last_payment_date = "{:%m-%d-%Y}".format(df.loc[last_payment, "Payment_Date"])
        
    # Truncate the data frame if we have additional principal payments:
    if addl_principal != 0:
                
        # Remove the extra payment periods
        df = df.loc[0:last_payment].copy()
        
        # Calculate the principal for the last row
        df.loc[last_payment, "Principal"] = -(df.loc[last_payment-1, "Curr_Balance"])
        
        # Calculate the total payment for the last row
        df.loc[last_payment, "Payment"] = df.loc[last_payment, ["Principal", "Interest"]].sum()
        
        # Zero out the additional principal
        df.loc[last_payment, "Addl_Principal"] = 0
        
    # Get the payment info into a DataFrame in column order
    payment_info = (df[["Payment", "Principal", "Addl_Principal", "Interest"]]
                    .sum().to_frame().T)
       
    # Format the Date DataFrame
    payment_details = pd.DataFrame.from_dict(dict([('payoff_date', [last_payment_date]),
                                               ('Interest Rate', [interest_rate]),
                                               ('Number of years', [years])
                                              ]))
    # Add a column showing how much we pay each period.
    # Combine addl principal with principal for total payment
    payment_details["Period_Payment"] = round(per_payment, 2) + addl_principal
    
    # Round the values
    df = df.round(2) 
    
    payment_summary = pd.concat([payment_details, payment_info], axis=1)
    return df, payment_summary

In [6]:
schedule1, stats1 = amortization_table(0.0361, 5, 12, 15000, addl_principal=0)
schedule1.to_excel(writer, sheet_name='Sheet1-SuperPrime')

schedule2, stats2 = amortization_table(0.0538, 5, 12, 15000, addl_principal=0)
schedule2.to_excel(writer, sheet_name='Sheet2-Prime')

schedule3, stats3 = amortization_table(0.0980, 5, 12, 15000, addl_principal=0)
schedule3.to_excel(writer, sheet_name='Sheet3-NonPrime')

schedule4, stats4 = amortization_table(0.1596, 5, 12, 15000, addl_principal=0)
schedule4.to_excel(writer, sheet_name='Sheet4-SubPrime')

schedule5, stats5 = amortization_table(0.1987, 5, 12, 15000, addl_principal=0)
schedule5.to_excel(writer, sheet_name='Sheet5-Deep SubPrime')

In [7]:
# Combine all the schedules into 1 view
df1 = pd.concat([schedule1, schedule2, schedule3, schedule4, schedule5], ignore_index=True)

df1['Start_Date'] = start_date
df1['Payment'] = df1['Payment'].abs()
df1['Principal'] = df1['Principal'].abs()
df1['Interest'] = df1['Interest'].abs()
df1['Cumulative_Principal'] = df1['Cumulative_Principal'].abs()

df1['Payment_Date'] = pd.to_datetime(df1['Payment_Date'], format='%Y-%m-%d')
df1['Start_Date'] = pd.to_datetime(df1['Start_Date'], format='%Y-%m-%d')


df1['Month'] = ((df1['Payment_Date'] - df1['Start_Date'])/np.timedelta64(1, 'M')) 
df1['Month'] = (df1['Month'].astype(int)) + 1
df1.to_excel(writer, sheet_name='Sheet6-CombinedList')
df1

,Payment_Date,Payment,Principal,Interest,Addl_Principal,Curr_Balance,Interest_Rate,Cumulative_Principal,Cumulative_Interest,Cumulative_Paid,PctOwed,PctPrinPaid,PrinipalPct,InterestPct,Start_Date,Month
0,2022-07-01,273.62,228.49,45.12,0,14771.51,3.61,228.49,45.12,273.61,98.48,1.52,83.51,16.49,2022-07-01,1
1,2022-08-01,273.62,229.18,44.44,0,14542.33,3.61,457.67,89.56,547.23,96.95,3.05,83.76,16.24,2022-07-01,2
2,2022-09-01,273.62,229.87,43.75,0,14312.46,3.61,687.54,133.31,820.85,95.42,4.58,84.01,15.99,2022-07-01,3
3,2022-10-01,273.62,230.56,43.06,0,14081.90,3.61,918.10,176.37,1094.47,93.88,6.12,84.26,15.74,2022-07-01,4
4,2022-11-01,273.62,231.25,42.36,0,13850.65,3.61,1149.35,218.73,1368.08,92.34,7.66,84.52,15.48,2022-07-01,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,2027-02-01,396.32,365.08,31.24,0,1521.78,19.87,13478.22,8715.94,22194.16,10.15,89.85,92.12,7.88,2022-07-01,56
296,2027-03-01,396.32,371.13,25.20,0,1150.65,19.87,13849.35,8741.14,22590.49,7.67,92.33,93.64,6.36,2022-07-01,56
297,2027-04-01,396.32,377.27,19.05,0,773.38,19.87,14226.62,8760.19,22986.81,5.16,94.84,95.19,4.81,2022-07-01,58
298,2027-05-01,396.32,383.52,12.81,0,389.86,19.87,14610.14,8773.00,23383.14,2.60,97.40,96.77,3.23,2022-07-01,58


In [8]:
# Combine all the scenarios into 1 view

stats1['Category'] = 'SuperPrime'
stats1['CSStart'] = '781'
stats1['CSStop'] = '850'
stats2['Category'] = 'Prime'
stats2['CSStart'] = '661'
stats2['CSStop'] = '780'
stats3['Category'] = 'NonPrime'
stats3['CSStart'] = '601'
stats3['CSStop'] = '660'
stats4['Category'] = 'SubPrime'
stats4['CSStart'] = '501'
stats4['CSStop'] = '600'
stats5['Category'] = 'Deep SubPrime'
stats5['CSStart'] = '300'
stats5['CSStop'] = '500'

df2 = pd.concat([stats1, stats2, stats3, stats4, stats5], ignore_index=True)

df2['Start_Date'] = start_date
df2["Interest Rate"] = df2["Interest Rate"] * 100
df2['Period_Payment'] = df2['Period_Payment'] * -1
df2['Payment'] = df2['Payment'] * -1
df2['Principal'] = df2['Principal'] * -1
df2['Interest'] = df2['Interest'] * -1

df2.to_excel(writer, sheet_name='Sheet7-RateComparison')
df2

,payoff_date,Interest Rate,Number of years,Period_Payment,Payment,Principal,Addl_Principal,Interest,Category,CSStart,CSStop,Start_Date
0,06-01-2027,3.61,5,273.62,16417.2,15000.02,0.0,1416.98,SuperPrime,781,850,2022-07-01
1,06-01-2027,5.38,5,285.69,17141.4,14999.98,0.0,2141.25,Prime,661,780,2022-07-01
2,06-01-2027,9.80,5,317.23,19033.8,15000.00,0.0,4033.91,NonPrime,601,660,2022-07-01
3,06-01-2027,15.96,5,364.45,21867.0,14999.98,0.0,6867.14,SubPrime,501,600,2022-07-01
4,06-01-2027,19.87,5,396.32,23779.2,15000.01,0.0,8779.46,Deep SubPrime,300,500,2022-07-01


In [12]:
Depreciation = pd.DataFrame()
Depreciation = df2[['Category', 'Interest Rate', 'Payment']]
Depreciation['Principal'] = 15000
Depreciation['Principal'] = Depreciation['Principal'].astype(float)
Depreciation['Yr1Paid'] = ((Depreciation['Payment'])/5)
Depreciation['Yr1Value'] = Depreciation['Principal'] * 0.81
Depreciation['Yr2Paid'] = ((Depreciation['Payment'])/5) * 2
Depreciation['Yr2Value'] = Depreciation['Principal'] * 0.69
Depreciation['Yr3Paid'] = ((Depreciation['Payment'])/5) * 3
Depreciation['Yr3Value'] = Depreciation['Principal'] * 0.58
Depreciation['Yr4Paid'] = ((Depreciation['Payment'])/5) * 4
Depreciation['Yr4Value'] = Depreciation['Principal'] * 0.49

Depreciation['Yr5Value'] = Depreciation['Principal'] * 0.4
Depreciation['Yr5Diff'] = Depreciation['Payment'] - Depreciation['Yr5Value']
Depreciation = Depreciation.set_index('Category')
Depreciation = Depreciation.apply(pd.to_numeric).round(2)
Depreciation.to_excel(writer, sheet_name='Sheet8-Depreciation')
print("After Depreciation Values: Year 1 (81%); Year 2 (69%); Year 3 (58%); Year 4 (49%); Year 5 (40%)")
print()
Depreciation

After Depreciation Values: Year 1 (81%); Year 2 (69%); Year 3 (58%); Year 4 (49%); Year 5 (40%)



<ipython-input-12-06ba9fd86fe1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Depreciation['Principal'] = 15000
<ipython-input-12-06ba9fd86fe1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Depreciation['Principal'] = Depreciation['Principal'].astype(float)
<ipython-input-12-06ba9fd86fe1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

,Interest Rate,Payment,Principal,Yr1Paid,Yr1Value,Yr2Paid,Yr2Value,Yr3Paid,Yr3Value,Yr4Paid,Yr4Value,Yr5Value,Yr5Diff
Category,,,,,,,,,,,,,
SuperPrime,3.61,16417.2,15000.0,3283.44,12150.0,6566.88,10350.0,9850.32,8700.0,13133.76,7350.0,6000.0,10417.2
Prime,5.38,17141.4,15000.0,3428.28,12150.0,6856.56,10350.0,10284.84,8700.0,13713.12,7350.0,6000.0,11141.4
NonPrime,9.80,19033.8,15000.0,3806.76,12150.0,7613.52,10350.0,11420.28,8700.0,15227.04,7350.0,6000.0,13033.8
SubPrime,15.96,21867.0,15000.0,4373.40,12150.0,8746.80,10350.0,13120.20,8700.0,17493.60,7350.0,6000.0,15867.0
Deep SubPrime,19.87,23779.2,15000.0,4755.84,12150.0,9511.68,10350.0,14267.52,8700.0,19023.36,7350.0,6000.0,17779.2


In [10]:
writer.save()